In [21]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow_hub as hub
import cv2
import numpy as np

# Load the pre-trained object detection model from TensorFlow Hub
object_detection_model = hub.load("https://tfhub.dev/tensorflow/ssd_mobilenet_v2/2")

# Define the paths to your dataset
train_data_dir = "D:/Foundathon/Fooddataset/Train"
test_data_dir = "D:/Foundathon/Fooddataset/Test"

# Image dimensions and batch size
img_width, img_height = 224, 224
batch_size = 16

# Create a data generator for training data
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

# Create a data generator for test data (no data augmentation)
test_datagen = ImageDataGenerator(rescale=1. / 255)

# Generate batches of augmented data for training
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Generate batches of test data
test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical'
)

# Build a simple convolutional neural network (CNN) for food classification
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(4, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=test_generator,
    validation_steps=test_generator.samples // batch_size
)

# Save the model
model.save('food_classifier_model.h5')

# Function to predict class and calorie count for an image
def predict_class_and_calories(image_path):
    img = cv2.imread(image_path)
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img_resized = cv2.resize(img_rgb, (img_width, img_height))
    img_normalized = img_resized / 255.0
    img_input = np.expand_dims(img_normalized, axis=0)

    # Predict class probabilities
    predictions = model.predict(img_input)[0]
    predicted_class = np.argmax(predictions)

    # Print predicted class and corresponding calorie count
    if predicted_class == 0:
        food_class = "Apple"
        calorie_count = 95  # Replace with the actual calorie count for an apple
    elif predicted_class == 1:
        food_class = "Banana"
        calorie_count = 105  # Replace with the actual calorie count for a banana
    elif predicted_class == 2:
        food_class = "Bread"
        calorie_count = 80  # Replace with the actual calorie count for bread
    elif predicted_class == 3:
        food_class = "Carrot"
        calorie_count = 41  # Replace with the actual calorie count for a carrot

    print(f"Predicted Class: {food_class}")
    print(f"Calorie Count: {calorie_count} calories")

# Replace 'path_to_test_image.jpg' with the path to an image from your test dataset
image_path = "C:\\Users\\grove\Downloads\\banana.jpg"
predict_class_and_calories(image_path)


Found 80 images belonging to 4 classes.
Found 36 images belonging to 4 classes.
Epoch 1/10
5/5 [==============================] - 18s 3s/step - loss: 2.0098 - accuracy: 0.2875 - val_loss: 1.3871 - val_accuracy: 0.2188
Epoch 2/10
5/5 [==============================] - 11s 2s/step - loss: 1.3551 - accuracy: 0.4000 - val_loss: 1.2541 - val_accuracy: 0.5938
Epoch 3/10
5/5 [==============================] - 11s 2s/step - loss: 1.1673 - accuracy: 0.5750 - val_loss: 0.8645 - val_accuracy: 0.8438
Epoch 4/10
5/5 [==============================] - 11s 2s/step - loss: 0.7211 - accuracy: 0.7875 - val_loss: 0.4041 - val_accuracy: 0.9375
Epoch 5/10
5/5 [==============================] - 11s 2s/step - loss: 0.4466 - accuracy: 0.8875 - val_loss: 0.2598 - val_accuracy: 0.9062
Epoch 6/10
5/5 [==============================] - 10s 2s/step - loss: 0.2737 - accuracy: 0.8750 - val_loss: 0.1204 - val_accuracy: 1.0000
Epoch 7/10
5/5 [==============================] - 11s 2s/step - loss: 0.1843 - accuracy: 0.9

In [ ]:
# Save the weights of the model to a new file
new_weights_filename = 'smallsizeclassifier.h5'
model.save_weights(new_weights_filename)
